In [2]:
import numpy as np
import pandas as pd

# Mengatur agar output tabel lebih rapi
pd.set_option('display.float_format', lambda x: '%.6f' % x)

# --- FUNGSI-FUNGSI PERSAMAAN ---

# Iterasi Titik Tetap (sesuai NIMx=2)
# g1B dari halaman 6
def g1B(x, y):
    val = 10 - x * y
    # Mencegah error jika nilai di dalam akar negatif
    if val < 0:
        return np.nan 
    return np.sqrt(val)

# g2A dari halaman 5
def g2A(x, y):
    return 57 - 3 * x * y**2

# Turunan Parsial untuk Metode Newton-Raphson (dari halaman 11)
def du_dx(x, y):
    return 2 * x + y

def du_dy(x, y):
    return x

def dv_dx(x, y):
    return 3 * y**2

def dv_dy(x, y):
    return 1 + 6 * x * y

# --- NILAI AWAL DAN PARAMETER ---

x0 = 1.5
y0 = 3.5
tolerance = 0.000001
max_iterations = 50 # Batas iterasi untuk mencegah loop tak hingga

print("Setup selesai. Semua fungsi dan variabel awal telah siap.")

Setup selesai. Semua fungsi dan variabel awal telah siap.


In [ ]:
print("--- 1. Metode Iterasi Titik Tetap (Jacobi) ---")

# Inisialisasi nilai awal dari setup
x_jacobi = x0
y_jacobi = y0

# List untuk menyimpan hasil setiap iterasi (termasuk delta)
results_jacobi = []

# Iterasi ke-0: sebelum ada perhitungan, Δx dan Δy = 0
results_jacobi.append([0, x_jacobi, y_jacobi, 0, 0])

# Proses iterasi metode Jacobi
for i in range(1, max_iterations):
    # Hitung nilai x dan y berikutnya menggunakan fungsi iterasi g1B dan g2A
    x_next = g1B(x_jacobi, y_jacobi)
    y_next = g2A(x_jacobi, y_jacobi)
    
    # Hitung perubahan (delta) dari nilai sebelumnya
    delta_x = abs(x_next - x_jacobi) if not np.isnan(x_next) else np.nan
    delta_y = abs(y_next - y_jacobi) if not np.isnan(y_next) else np.nan
    
    # Simpan hasil iterasi ke dalam list
    results_jacobi.append([i, x_next, y_next, delta_x, delta_y])
    
    # Cek apakah hasilnya divergen (menjadi NaN atau tak terhingga)
    if np.isnan(x_next) or np.isnan(y_next) or np.isinf(x_next) or np.isinf(y_next):
        print(f"WARNING: Iterasi ke-{i}: Solusi *divergen* karena menghasilkan nilai tidak valid.")
        break
    
    # Cek kondisi berhenti jika sudah konvergen (perubahan lebih kecil dari toleransi)
    if delta_x < tolerance and delta_y < tolerance:
        print(f"SUCCESS: Konvergen pada iterasi ke-{i}.")
        break
    
    # Perbarui nilai untuk iterasi selanjutnya
    x_jacobi = x_next
    y_jacobi = y_next

# Jika loop selesai tanpa 'break', berarti tidak konvergen dalam jumlah iterasi maksimum
else:
    print(f"FAIL: Metode tidak konvergen dalam {max_iterations} iterasi.")

# Buat DataFrame hasil iterasi (tetap menggunakan NaN jika ada)
df_jacobi = pd.DataFrame(results_jacobi, columns=['Iterasi', 'x', 'y', 'Δx', 'Δy'])

# Tampilkan hasil iterasi dalam bentuk tabel
print("\n Hasil Iterasi Jacobi:")
display(df_jacobi)


--- 1. Metode Iterasi Titik Tetap (Jacobi) ---

 Hasil Iterasi Jacobi:


,Iterasi,x,y,Δx,Δy
0,0,1.500000,3.500000,0.000000,0.000000
1,1,2.179449,1.875000,0.679449,1.625000
2,2,2.431776,34.013619,0.252326,32.138619
3,3,NaN,-8383.154970,NaN,8417.168589


In [9]:
print("--- 2. Metode Iterasi Titik Tetap (Seidel) ---")

# Inisialisasi nilai awal dari setup
x_seidel = x0
y_seidel = y0

# List untuk menyimpan hasil setiap iterasi (termasuk delta)
results_seidel = []

# Iterasi ke-0: sebelum ada perhitungan, Δx dan Δy = 0
results_seidel.append([0, x_seidel, y_seidel, 0, 0])

# Proses iterasi metode Seidel
for i in range(1, max_iterations):
    # Hitung nilai x berikutnya menggunakan nilai saat ini
    x_next = g1B(x_seidel, y_seidel)
    
    # Jika x_next tidak valid, hentikan iterasi
    if np.isnan(x_next) or np.isinf(x_next):
        print(f"WARNING: Iterasi ke-{i}: Solusi divergen karena nilai x tidak valid.")
        results_seidel.append([i, x_next, np.nan, np.nan, np.nan])
        break
    
    # Hitung nilai y berikutnya menggunakan x yang baru
    y_next = g2A(x_next, y_seidel)
    
    # Jika y_next tidak valid, hentikan iterasi
    if np.isnan(y_next) or np.isinf(y_next):
        print(f"WARNING: Iterasi ke-{i}: Solusi divergen karena nilai y tidak valid.")
        results_seidel.append([i, x_next, y_next, np.nan, np.nan])
        break
    
    # Hitung perubahan (delta) dari iterasi sebelumnya
    delta_x = abs(x_next - x_seidel) if not np.isnan(x_next) else np.nan
    delta_y = abs(y_next - y_seidel) if not np.isnan(y_next) else np.nan
    
    # Simpan hasil iterasi ke dalam list
    results_seidel.append([i, x_next, y_next, delta_x, delta_y])
    
    # Cek kondisi berhenti jika sudah konvergen (Δ lebih kecil dari toleransi)
    if delta_x < tolerance and delta_y < tolerance:
        print(f"SUCCESS: Konvergen pada iterasi ke-{i}.")
        break
    
    # Perbarui nilai untuk iterasi selanjutnya
    x_seidel = x_next
    y_seidel = y_next

# Jika loop selesai tanpa 'break', berarti tidak konvergen dalam jumlah iterasi maksimum
else:
    print(f"FAIL: Metode tidak konvergen dalam {max_iterations} iterasi.")

# Buat DataFrame hasil iterasi (tetap menggunakan NaN jika ada)
df_seidel = pd.DataFrame(results_seidel, columns=['Iterasi', 'x', 'y', 'Δx', 'Δy'])

# Tampilkan hasil iterasi dalam bentuk tabel
print("\n Hasil Iterasi Seidel:")
display(df_seidel)


--- 2. Metode Iterasi Titik Tetap (Seidel) ---

 Hasil Iterasi Seidel:


C:\Users\faiz\AppData\Local\Temp\ipykernel_10124\4128386921.py:20: RuntimeWarning: overflow encountered in scalar power
  return 57 - 3 * x * y**2


,Iterasi,x,y,Δx,Δy
0,0,1.500000,3.500000,0.000000,0.000000
1,1,2.179449,-23.094768,0.679449,26.594768
2,2,7.767489,-12371.796972,5.588039,12348.702203
3,3,310.012569,-142352836336.902557,302.245080,142352823965.105591
4,4,6643129.416388,-403855700441468460062893670400.000000,6642819.403819,403855700441468460062893670400.000000
5,5,1637945567953550336.000000,-8014439496856225745992880355390086154760446671...,1637945567946907136.000000,80144394968562257459928803553900861547604466713...
6,6,11457406187051036650452770696635850501124081582...,-2207770235159068671448928442632796734368672984...,11457406187051036650452770696635850501124081582...,22077702351590686714489284426327967343686729848...
7,7,15904502617780485701775373049311129299808302362...,-inf,NaN,NaN


In [15]:
print("--- 3. Metode Newton-Raphson ---")

# Menggunakan nilai awal dari setup
x_nr = x0
y_nr = y0

# List untuk menyimpan hasil setiap iterasi (termasuk delta)
results_nr = []

# Iterasi ke-0: sebelum ada perhitungan, Δx dan Δy = 0
results_nr.append([0, x_nr, y_nr, 0, 0])

# Proses iterasi Newton-Raphson
for i in range(1, max_iterations):
    # Hitung nilai fungsi u dan v pada titik (x, y) saat ini
    u = x_nr**2 + x_nr * y_nr - 10
    v = y_nr + 3 * x_nr * y_nr**2 - 57

    # Cek kondisi berhenti: jika nilai u dan v sudah sangat dekat dengan 0
    if abs(u) < tolerance and abs(v) < tolerance:
        print(f"SUCCESS: Konvergen pada iterasi ke-{i}.")
        break

    # Hitung determinan matriks Jacobi
    J = du_dx(x_nr, y_nr) * dv_dy(x_nr, y_nr) - du_dy(x_nr, y_nr) * dv_dx(x_nr, y_nr)
    
    # Cegah pembagian dengan nol jika determinan terlalu kecil
    if abs(J) < 1e-10:
        print("STOPPED: Determinan Jacobi mendekati nol. Metode dihentikan.")
        results_nr.append([i, np.nan, np.nan, np.nan, np.nan])
        break

    # Hitung nilai x dan y berikutnya menggunakan formula Newton-Raphson
    x_next = x_nr - (u * dv_dy(x_nr, y_nr) - v * du_dy(x_nr, y_nr)) / J
    y_next = y_nr - (v * du_dx(x_nr, y_nr) - u * dv_dx(x_nr, y_nr)) / J

    # Hitung perubahan (delta) dari iterasi sebelumnya
    delta_x = abs(x_next - x_nr) if not np.isnan(x_next) else np.nan
    delta_y = abs(y_next - y_nr) if not np.isnan(y_next) else np.nan

    # Simpan hasil iterasi ke dalam list
    results_nr.append([i, x_next, y_next, delta_x, delta_y])

    # Perbarui nilai untuk iterasi berikutnya
    x_nr = x_next
    y_nr = y_next

# Jika loop selesai tanpa 'break', berarti tidak konvergen dalam jumlah iterasi maksimum
else:
    print(f"FAIL: Metode tidak konvergen dalam {max_iterations} iterasi.")

# Buat DataFrame hasil iterasi (tetap menggunakan NaN jika ada)
df_nr = pd.DataFrame(results_nr, columns=['Iterasi', 'x', 'y', 'Δx', 'Δy'])

# Tampilkan hasil iterasi dalam bentuk tabel
print("\n Hasil Iterasi Newton-Raphson:")
display(df_nr)


--- 3. Metode Newton-Raphson ---
SUCCESS: Konvergen pada iterasi ke-5.

 Hasil Iterasi Newton-Raphson:


,Iterasi,x,y,Δx,Δy
0,0,1.500000,3.500000,0.000000,0.000000
1,1,2.036029,2.843875,0.536029,0.656125
2,2,1.998701,3.002289,0.037328,0.158413
3,3,2.000000,2.999999,0.001299,0.002289
4,4,2.000000,3.000000,0.000000,0.000001


In [ ]:
print("--- 4. Metode Secant (Metode Broyden) untuk Sistem Persamaan ---")

# Menggunakan nilai awal dari setup dalam bentuk array/vektor
x_secant = np.array([x0, y0], dtype=float)

# Inisialisasi Matriks Jacobi (B) secara numerik
h = 1e-6
J00 = (((x0+h)**2 + (x0+h)*y0 - 10) - (x0**2 + x0*y0 - 10)) / h
J01 = ((x0**2 + x0*(y0+h) - 10) - (x0**2 + x0*y0 - 10)) / h
J10 = (((y0 + 3*(x0+h)*y0**2 - 57)) - (y0 + 3*x0*y0**2 - 57)) / h
J11 = (((y0+h) + 3*x0*(y0+h)**2 - 57) - (y0 + 3*x0*y0**2 - 57)) / h
B = np.array([[J00, J01], [J10, J11]], dtype=float)

# List untuk menyimpan hasil iterasi (termasuk Δx dan Δy)
results_secant = []

# Simpan iterasi ke-0 (nilai awal, Δ = 0)
results_secant.append([0, x_secant[0], x_secant[1], 0.0, 0.0])

for i in range(1, max_iterations + 1):
    # Hitung nilai fungsi pada titik saat ini
    F_current = np.array([
        x_secant[0]**2 + x_secant[0]*x_secant[1] - 10,
        x_secant[1] + 3*x_secant[0]*x_secant[1]**2 - 57
    ])
    
    # Cek kondisi berhenti
    if np.linalg.norm(F_current) < tolerance:
        print(f"SUCCESS: Konvergen pada iterasi ke-{i-1}.")
        break

    # Selesaikan sistem linear B * delta_x = -F
    try:
        delta_x = np.linalg.solve(B, -F_current)
    except np.linalg.LinAlgError:
        print("STOPPED: Matriks singular. Metode dihentikan.")
        break
    
    # Hitung titik berikutnya
    x_next = x_secant + delta_x
    
    # Hitung Δx dan Δy
    delta_x_val = x_next[0] - x_secant[0]
    delta_y_val = x_next[1] - x_secant[1]
    
    # Tangani hasil yang tidak valid (NaN atau inf)
    if np.isnan(x_next[0]) or np.isnan(x_next[1]) or np.isinf(x_next[0]) or np.isinf(x_next[1]):
        results_secant.append([i, np.nan, np.nan, np.nan, np.nan])
        print(f"FAIL: Solusi divergen pada iterasi ke-{i}.")
        break
    
    # Simpan hasil iterasi
    results_secant.append([i, x_next[0], x_next[1], delta_x_val, delta_y_val])
    
    # Hitung nilai fungsi pada titik baru
    F_next = np.array([
        x_next[0]**2 + x_next[0]*x_next[1] - 10,
        x_next[1] + 3*x_next[0]*x_next[1]**2 - 57
    ])

    # Update matriks B menggunakan formula Broyden
    y_k = F_next - F_current
    B += np.outer((y_k - B @ delta_x), delta_x) / (delta_x @ delta_x)
    
    # Perbarui nilai untuk iterasi selanjutnya
    x_secant = x_next
else:
    print(f"FAIL: Metode tidak konvergen dalam {max_iterations} iterasi.")

# Tampilkan hasil iterasi dalam bentuk tabel menggunakan display()
df_secant = pd.DataFrame(results_secant, columns=['Iterasi', 'x', 'y', 'Δx', 'Δy'])

print("\nHasil Iterasi Secant (Broyden):")
display(df_secant)


--- 4. Metode Secant (Metode Broyden) untuk Sistem Persamaan ---
SUCCESS: Konvergen pada iterasi ke-6.

Hasil Iterasi Secant (Broyden):


,Iterasi,x,y,Δx,Δy
0,0,1.500000,3.500000,0.000000,0.000000
1,1,2.036029,2.843875,0.536029,-0.656125
2,2,2.008919,2.997379,-0.027110,0.153504
3,3,1.998308,3.002865,-0.010610,0.005485
4,4,1.999841,3.000268,0.001533,-0.002597
5,5,2.000000,3.000000,0.000159,-0.000268
6,6,2.000000,3.000000,-0.000000,0.000000
